In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
# sys.path.append('/uni-mainz.de/homes/odoensen/Work/Scripts/')
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
import matplotlib.ticker as mtick
from glob import glob
import matplotlib

%matplotlib qt

In [2]:
#set path
# path = '/uni-mainz.de/homes/odoensen/Work/T850/GSS/'
path = '/media/onno/Volume/GFS_T850/GSS/'

Smoothed GSS for entire European Continent

In [4]:
#if True use all adjecent grid boxes to calculate spatially averaged GSS over a total of 9 grid boxes
smoothing=True
from mpl_toolkits.basemap import Basemap
#Define relevant lead days
lead_dayz = [5]
#define coordinates of left and right, up and down edge of every box to read the file
lonz_0 = np.arange(-32,56.1,4,dtype=np.int64)
lonz_1 = np.arange(-28,60.1,4,dtype=np.int64)
latz_1 = np.arange(30,74.1,4,dtype=np.int64)
latz_0 = np.arange(34,78.1,4,dtype=np.int64)
#convert negative longitude coordinates to positive ones by adding 360 degrees
lonz_0 = [int(i+360) if i<0 else int(i) for i in lonz_0]
lonz_1 = [int(i+360) if i<=0 else int(i) for i in lonz_1]

#Season with month numbers
seasonz = {'MAM':[3,4,5],
          'JJA':[6,7,8],
          'SON':[9,10,11],
          'DJF':[12,1,2]}
#Define model
modelz = ['ERA5RF']

month_list = ['DJF','JJA','DJF','JJA']

extremez_list = ['persistent_hw','persistent_hw',
                'persistent_cw','persistent_cw']

extreme_labelz = ['Warm','Warm','Cold','Cold']
GSS_file = "Monthly_mean_GSS_lead_day_5_lon_{}_{}_lat_{}_{}_{}.csv"
#Define color map
colorz = [(0,'#ff0000'),(0.25,'#ff6600'),(0.5,'#ffff00'),
          (0.7,'#33cc33'),(1,'#006600')]   
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('custom', colorz, N=1024)
letterz=['a','b','c','d']

#Define coordinates to plot. These are 1 greater in dimension than the previois defined coordinates so plt.pcolormesh can use them properly
plot_lonz = np.arange(-32,60.1,4)
plot_latz = np.arange(30,78.1,4)

#Convert to 2D grid
plot_lonz_2D,plot_latz_2D = np.meshgrid(plot_lonz,plot_latz)
#Loop over different modelz
for model in modelz:
    #Loop over all seasons
    fig,axz=plt.subplots(2,2,figsize=(8,5.25))
    for i in range(len(month_list)):
        ax = axz.flat[i]
        ax.set_title('{} Extreme {}'.format(extreme_labelz[i],month_list[i]))
        ax.text(x=-0,y=1.125,s=letterz[i]+')',transform=ax.transAxes,fontweight='bold',fontsize=13,
                       verticalalignment='top')
        #Create empty array to fill with GSS values later
        GSS_europe = np.empty((len(latz_0),len(lonz_1)))
        #Pick subsequent labels
        month = month_list[i]
        extreme = extremez_list[i]
        #Fill empty array with GSS values
        for x in range(len(latz_0)):
            for y in range(len(lonz_0)):
                lat_0=latz_0[::-1][x];lat_1=latz_1[::-1][x]
                lon_0=lonz_0[y];lon_1=lonz_1[y]
                df = pd.read_csv(path+GSS_file.format(lon_0,lon_1,lat_1,lat_0,model),index_col=0)
                GSS = df.loc[month][extreme]
                GSS_europe[x,y]=GSS
        #If smooothing is True, smooth array using spatial average
        if smoothing:
            GSS_europe = my_tools.average_2D_matrix(GSS_europe)
#             np.save(path+'/figures/map_europe_GSS_{}_{}_day_5_{}_smoothed.npy'.format(extreme,month,lead_day,model),
#                GSS_europe)
        #intializing map of Europe
        m = Basemap(llcrnrlon=-28,llcrnrlat=34,urcrnrlon=56,urcrnrlat=74,
                     resolution='i', projection='cyl', lat_0 = 52.5, lon_0 = 15,ax=ax)

        m.drawmapboundary(fill_color='w')
#             m.fillcontinents(color='w',lake_color='w')
        m.drawcoastlines(linewidth=.5)
        m.drawcountries(linewidth=.5)
        parallels = np.arange(0.,81,10.)
        # labels = [left,right,top,bottom]
        meridians = np.arange(0,361.,20.)
        m.drawparallels(parallels)
        m.drawmeridians(meridians)
        if i%2!=0:
            m.drawparallels(parallels,labels=[False,True,False,False],fontsize=8)
        if i>=2:
            m.drawmeridians(meridians,labels=[False,False,False,True],fontsize=8)
        #Plotting data
        im = m.pcolormesh(plot_lonz_2D,plot_latz_2D,GSS_europe,latlon='true',
                          cmap=cmap,vmin=0.2,vmax=0.8)
    #Further format figure
    fig.subplots_adjust(left=0.025,right=0.95,bottom=0.125,top=0.9,wspace=0.05,hspace=0.02)
    cax = fig.add_axes([0.025, 0.09, 0.925, 0.025])
    fig.colorbar(im, cax=cax, orientation='horizontal',label='Gilbert Skill Score')
    fig.suptitle('Gilbert Skill Score Europe: Day 5 Forecast {}'.format(model),fontsize=15)
    if smoothing:
        fig.savefig('/media/onno/Volume/Bern/GSS_Europe_JJA_DJF_smoothed.png',dpi=250)
    else:
        fig.savefig('/media/onno/Volume/Bern/GSS_Europe_JJA_DJF.png',dpi=250)
    plt.close(fig)

In [4]:
#if True use all adjecent grid boxes to calculate spatially averaged GSS over a total of 9 grid boxes
smoothing=True
from mpl_toolkits.basemap import Basemap
#Define relevant lead days
lead_dayz = [5]
#define coordinates of left and right, up and down edge of every box to read the file
lonz_0 = np.arange(-32,56.1,4,dtype=np.int64)
lonz_1 = np.arange(-28,60.1,4,dtype=np.int64)
latz_1 = np.arange(30,74.1,4,dtype=np.int64)
latz_0 = np.arange(34,78.1,4,dtype=np.int64)
#convert negative longitude coordinates to positive ones by adding 360 degrees
lonz_0 = [int(i+360) if i<0 else int(i) for i in lonz_0]
lonz_1 = [int(i+360) if i<=0 else int(i) for i in lonz_1]

#Season with month numbers
seasonz = {'MAM':[3,4,5],
          'JJA':[6,7,8],
          'SON':[9,10,11],
          'DJF':[12,1,2]}
#Define model
modelz = ['ERA5RF']

month_list = ['DJF','JJA','DJF','JJA']

extremez_list = ['persistent_hw','persistent_hw',
                'persistent_cw','persistent_cw']

extreme_labelz = ['Warm','Warm','Cold','Cold']
GSS_file = "Monthly_mean_GSS_lead_day_5_lon_{}_{}_lat_{}_{}_{}.csv"
#Define color map
colorz = [(0,'#ff0000'),(0.25,'#ff6600'),(0.5,'#ffff00'),
          (0.7,'#33cc33'),(1,'#006600')]   
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('custom', colorz, N=1024)
letterz=['a','b','c','d']

#Define coordinates to plot. These are 1 greater in dimension than the previois defined coordinates so plt.pcolormesh can use them properly
plot_lonz = np.arange(-32,60.1,4)
plot_latz = np.arange(30,78.1,4)

#Convert to 2D grid
plot_lonz_2D,plot_latz_2D = np.meshgrid(plot_lonz,plot_latz)
#Loop over different modelz
for model in modelz:
    for x
    #Loop over all seasons
    fig,axz=plt.subplots(4,3,figsize=(10,12))
    for i in range(len(month_list)):
        ax = axz.flat[i]
        ax.set_title('{} Extreme {}'.format(extreme_labelz[i],month_list[i]))
        ax.text(x=-0,y=1.125,s=letterz[i]+')',transform=ax.transAxes,fontweight='bold',fontsize=13,
                       verticalalignment='top')
        #Create empty array to fill with GSS values later
        GSS_europe = np.empty((len(latz_0),len(lonz_1)))
        #Pick subsequent labels
        month = month_list[i]
        extreme = extremez_list[i]
        #Fill empty array with GSS values
        for x in range(len(latz_0)):
            for y in range(len(lonz_0)):
                lat_0=latz_0[::-1][x];lat_1=latz_1[::-1][x]
                lon_0=lonz_0[y];lon_1=lonz_1[y]
                df = pd.read_csv(path+GSS_file.format(lon_0,lon_1,lat_1,lat_0,model),index_col=0)
                GSS = df.loc[month][extreme]
                GSS_europe[x,y]=GSS
        #If smooothing is True, smooth array using spatial average
        if smoothing:
            GSS_europe = my_tools.average_2D_matrix(GSS_europe)
#             np.save(path+'/figures/map_europe_GSS_{}_{}_day_5_{}_smoothed.npy'.format(extreme,month,lead_day,model),
#                GSS_europe)
        #intializing map of Europe
        m = Basemap(llcrnrlon=-28,llcrnrlat=34,urcrnrlon=56,urcrnrlat=74,
                     resolution='i', projection='cyl', lat_0 = 52.5, lon_0 = 15,ax=ax)

        m.drawmapboundary(fill_color='w')
#             m.fillcontinents(color='w',lake_color='w')
        m.drawcoastlines(linewidth=.5)
        m.drawcountries(linewidth=.5)
        parallels = np.arange(0.,81,10.)
        # labels = [left,right,top,bottom]
        meridians = np.arange(0,361.,20.)
        m.drawparallels(parallels)
        m.drawmeridians(meridians)
        if i%2!=0:
            m.drawparallels(parallels,labels=[False,True,False,False],fontsize=8)
        if i>=2:
            m.drawmeridians(meridians,labels=[False,False,False,True],fontsize=8)
        #Plotting data
        im = m.pcolormesh(plot_lonz_2D,plot_latz_2D,GSS_europe,latlon='true',
                          cmap=cmap,vmin=0.2,vmax=0.8)
    #Further format figure
    fig.subplots_adjust(left=0.025,right=0.95,bottom=0.125,top=0.9,wspace=0.05,hspace=0.02)
    cax = fig.add_axes([0.025, 0.09, 0.925, 0.025])
    fig.colorbar(im, cax=cax, orientation='horizontal',label='Gilbert Skill Score')
    fig.suptitle('Gilbert Skill Score Europe: Day 5 Forecast {}'.format(model),fontsize=15)
    if smoothing:
        fig.savefig('/media/onno/Volume/Bern/GSS_Europe_JJA_DJF_smoothed.png',dpi=250)
    else:
        fig.savefig('/media/onno/Volume/Bern/GSS_Europe_JJA_DJF.png',dpi=250)
    plt.close(fig)